In [ ]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

sys.path.append(os.getenv("ROOT"))

In [ ]:
import polars as pl
import pandas as pd
from scipy.stats import skew, kurtosis
import statsmodels.api as sm
import numpy as np
from datetime import date
from silverfund.datasets.crsp_daily import CRSPDaily
from silverfund.datasets.barra_specific_returns import BarraSpecificReturns
from silverfund.datasets.crsp_monthly import CRSPMonthly
from silverfund.datasets.master_monthly import MasterMonthly
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# start = date(1995, 7, 31)
start = date(2006, 1, 1)
end = date(2024, 12, 31)

## Testing economic theory behind reversal.
 - Use monthly trading volume as proxy for news events to inform reversal strength.


In [ ]:
crsp_monthly = (
    CRSPMonthly(start_date=start, end_date=end)
    .load_all()
    .select(["permno", "date", "prc", "ret", "vol"])
)

crsp_monthly

In [ ]:
crsp_monthly = crsp_monthly.with_columns(pl.col("ret").shift(1).over("permno").alias("rev"))
crsp_monthly = crsp_monthly.with_columns(
    pl.col("vol").pct_change(1).over("permno").alias("vol_diff")
)

crsp_monthly = crsp_monthly.drop_nulls()
# crsp_monthly = crsp_monthly.with_columns(
#     ((pl.col("rev") - pl.col("rev").mean()) / pl.col("rev").std()).over("date").alias("rev_score")
# )

crsp_monthly = crsp_monthly.with_columns(
    ((pl.col("rev").mean() - pl.col("rev")) / pl.col("rev").std()).over("date").alias("rev_score")
)


df = crsp_monthly.to_pandas()
df

In [ ]:
def calc_beta_and_tstat(x):
    beta = np.cov(x["vol_diff"], x["rev_score"])[0, 1] / np.var(x["vol_diff"])

    # Compute standard error of beta and t-stat, from perplexity
    se_beta = np.sqrt(np.var(x["rev_score"]) / (len(x) - 1)) / np.sqrt(np.var(x["vol_diff"]))
    t_stat = beta / se_beta
    return pd.Series({"beta": beta, "t_stat": t_stat})


beta = df.groupby(["date"]).apply(calc_beta_and_tstat).reset_index()
beta.index = beta["date"]
beta = beta[["beta", "t_stat"]]
beta

In [ ]:
plt.hist(beta["beta"], bins=30)
plt.title("Regression Coefficient Histogram (vol on reversal)")
# sns.kdeplot(beta['beta'], bw_adjust=1.5)
plt.show()

In [ ]:
plt.hist(beta["t_stat"], bins=30)
plt.title("Regression Coefficient T-stat Histogram (vol on reversal)")
# sns.kdeplot(beta['t_stat'], bw_adjust=10)
plt.show()

## Testing Barra residual short term mean reversal.
 - 


In [ ]:
crsp_monthly = (
    CRSPMonthly(start_date=start, end_date=end)
    .load_all()
    .select(["permco", "date", "prc", "ret", "vol"])
)

# reversal signals
crsp_monthly = crsp_monthly.with_columns(pl.col("ret").shift(1).over("permco").alias("rev"))
crsp_monthly = crsp_monthly.with_columns(pl.col("prc").shift(1).over("permco").alias("prclag"))
crsp_monthly = crsp_monthly.filter(pl.col("prclag") > 10)
crsp_monthly = crsp_monthly.drop_nulls(subset=["rev"])
crsp_monthly

In [ ]:
crsp_daily = (
    CRSPDaily(start_date=start, end_date=end)
    .load_all()
    .select(["permco", "date", "prc", "ret", "vol"])
)

crsp_daily = crsp_daily.to_pandas()
crsp_daily["mdt"] = crsp_daily["date"].values.astype("datetime64[M]")
crsp_daily["rolling_kurtosis"] = (
    crsp_daily.groupby("permco")["ret"]
    .rolling(window=22, min_periods=4)
    .kurt()
    .reset_index(level="permco", drop=True)
)
# check what type of kurt


def score_kurtosis(group):
    mean = group["rolling_kurtosis"].mean()
    std = group["rolling_kurtosis"].std()
    group["kurtosis_score"] = (group["rolling_kurtosis"] - mean) / std
    return group


crsp_daily = (
    crsp_daily.groupby("permco").apply(score_kurtosis).reset_index(level="permco", drop=True)
)
crsp_daily = crsp_daily.reset_index(drop=True)
crsp_daily = crsp_daily.groupby(["mdt", "permco"])["kurtosis_score"].max().reset_index()
crsp_daily = crsp_daily.reset_index(drop=True)
crsp_daily

In [ ]:
# crsp_daily = crsp_daily.to_pa
crsp_monthly = crsp_monthly.to_pandas()

crsp_monthly["month_year"] = crsp_monthly["date"].dt.strftime("%Y-%m")
crsp_daily["month_year"] = crsp_daily["mdt"].dt.strftime("%Y-%m")


# df = pd.merge(crsp_monthly, crsp_daily, on='month_year').drop(columns='month_year')
# df
crsp_monthly

In [ ]:
crsp_daily

In [ ]:
df = pd.merge(crsp_monthly, crsp_daily, on=["month_year", "permco"])
df

In [ ]:
df["kurt_lag"] = df.groupby("permco")["kurtosis_score"].shift(periods=1)
df.dropna(inplace=True)
df["rev_kurt"] = df["rev"] * df["kurt_lag"]
X = pd.concat([df["rev"], df["rev_kurt"]], axis=1)
X = sm.add_constant(X)
model = sm.OLS(df["ret"], X).fit()
print(model.summary())

In [ ]:
# previous work trying to filter
# crsp_monthly = crsp_monthly.with_columns(
#     pl.col("vol")
#     .rolling_mean(window_size=12)
#     .over("permco")
#     .alias("rolling_mean")
# )

# crsp_monthly = crsp_monthly.with_columns(
#     pl.col("vol")
#     .rolling_std(window_size=12)
#     .over("permco")
#     .alias("rolling_std")
# )

# crsp_monthly = crsp_monthly.with_columns((pl.col("rolling_mean") / pl.col("rolling_std")).over("permco").alias("vol_score"))
# crsp_monthly = crsp_monthly.drop_nulls(subset=["rolling_mean"])
# crsp_monthly = crsp_monthly.filter(pl.col("vol_score") <= 2)

In [ ]:
df = pl.DataFrame(df)
df

In [ ]:
df2 = df.clone()
df2 = df2.filter(pl.col("kurtosis_score") <= 2.5)

In [ ]:
labels = [str(x) for x in range(10)]


# crsp_monthly = crsp_monthly.with_columns(pl.col("rev").qcut(10, labels=labels, allow_duplicates=True).over("date").alias("bin"))
df = df.with_columns(
    pl.col("rev").qcut(10, labels=labels, allow_duplicates=True).over("date").alias("bin")
)

port = df.group_by(["date", "bin"]).agg(pl.col("ret").mean())
# port = crsp_monthly.group_by(["date", "bin"]).agg(pl.col("ret").mean())

port = port.pivot(on="bin", index="date", values="ret")
port = port.select(["date"] + labels)
port = port.sort(by="date")
port = port.with_columns((pl.col("0") - pl.col("9")).alias("spread"))
port = port.unpivot(index="date", variable_name="bin", value_name="ret")
port = port.sort(by=["date", "bin"])
port
result = port.group_by("bin").agg(
    [
        pl.col("ret").mean().cast(pl.Float64).alias("mean"),
        pl.col("ret").std().cast(pl.Float64).alias("std"),
        pl.col("ret").count().cast(pl.Float64).alias("count"),
    ]
)


result = result.with_columns(
    ((pl.col("mean") * 12) / (pl.col("std") * np.sqrt(12)))
    .cast(pl.Float64)
    .alias("annualized_sharpe")
)


result = result.with_columns(
    (pl.col("mean") / (pl.col("std") / pl.col("count").sqrt())).cast(pl.Float64).alias("tstat")
)


result = result.sort(by="bin")

result = result.transpose(include_header=True, column_names="bin", header_name="statistic")

print("Monthly results")
result

In [ ]:
port = port.with_columns(pl.col("ret").log1p().over("bin").alias("logret"))
port = port.with_columns(pl.col("logret").cum_sum().over("bin").alias("cumret") * 100)

plt.figure(figsize=(10, 6))
sns.lineplot(
    port.filter(pl.col("bin").is_in(["0", "9", "spread"])), x="date", y="cumret", hue="bin"
)

plt.ylabel("Cummulative Sum Return (%)")
plt.show()

In [ ]:
labels = [str(x) for x in range(10)]


# crsp_monthly = crsp_monthly.with_columns(pl.col("rev").qcut(10, labels=labels, allow_duplicates=True).over("date").alias("bin"))
df2 = df2.with_columns(
    pl.col("rev").qcut(10, labels=labels, allow_duplicates=True).over("date").alias("bin")
)

port = df2.group_by(["date", "bin"]).agg(pl.col("ret").mean())
# port = crsp_monthly.group_by(["date", "bin"]).agg(pl.col("ret").mean())

port = port.pivot(on="bin", index="date", values="ret")
port = port.select(["date"] + labels)
port = port.sort(by="date")
port = port.with_columns((pl.col("0") - pl.col("9")).alias("spread"))
port = port.unpivot(index="date", variable_name="bin", value_name="ret")
port = port.sort(by=["date", "bin"])
port
result = port.group_by("bin").agg(
    [
        pl.col("ret").mean().cast(pl.Float64).alias("mean"),
        pl.col("ret").std().cast(pl.Float64).alias("std"),
        pl.col("ret").count().cast(pl.Float64).alias("count"),
    ]
)


result = result.with_columns(
    ((pl.col("mean") * 12) / (pl.col("std") * np.sqrt(12)))
    .cast(pl.Float64)
    .alias("annualized_sharpe")
)


result = result.with_columns(
    (pl.col("mean") / (pl.col("std") / pl.col("count").sqrt())).cast(pl.Float64).alias("tstat")
)


result = result.sort(by="bin")

result = result.transpose(include_header=True, column_names="bin", header_name="statistic")

print("Monthly results")
result

In [ ]:
port = port.with_columns(pl.col("ret").log1p().over("bin").alias("logret"))
port = port.with_columns(pl.col("logret").cum_sum().over("bin").alias("cumret") * 100)

plt.figure(figsize=(10, 6))
sns.lineplot(
    port.filter(pl.col("bin").is_in(["0", "9", "spread"])), x="date", y="cumret", hue="bin"
)

plt.ylabel("Cummulative Sum Return (%)")
plt.show()

In [ ]:
# master = MasterMonthly(start_date=start, end_date=end, quiet=False).load_all().to_pandas()
master = MasterMonthly(start_date=start, end_date=end, quiet=False).load_all()
master

In [ ]:
df = master.clone()
df

In [ ]:
df = master.clone()
# reversal signals
df = df.with_columns(pl.col("log_spec_ret").shift(1).over("barrid").alias("barra_rev"))
df = df.with_columns(pl.col("ret").shift(1).over("barrid").alias("rev"))
df = df.with_columns(pl.col("price").shift(1).over("barrid").alias("prclag"))
df = df.filter(pl.col("prclag") > 5)
df = df.drop_nulls(subset=["rev"])
df = df.drop_nulls(subset=["barra_rev"])


df = df.with_columns(
    pl.col("log_spec_ret").rolling_skew(window_size=6).over("barrid").alias("rolling_skew")
)

# pandas_df = df.to_pandas()
# pandas_df['rolling_kurtosis'] = pandas_df.groupby('barrid')['log_spec_ret'].rolling(
#     window=12, min_periods=4).kurt()
# df = pl.from_pandas(pandas_df)


df = df.with_columns(pl.col("rolling_skew").quantile(0.90).over(["barrid"]).alias("skew_90"))

df = df.drop_nulls(subset=["rolling_skew"])
df = df.drop_nulls(subset=["skew_90"])

# df = df.filter(pl.col("rolling_skew") <= pl.col("skew_90"))

df = df.filter(np.abs(pl.col("rolling_skew")) < 1)
print(len(df))
df

In [ ]:
labels = [str(x) for x in range(10)]

df = df.with_columns(pl.col("rev").qcut(10, labels=labels).over("date").alias("bin"))
df = df.with_columns(pl.col("barra_rev").qcut(10, labels=labels).over("date").alias("barra_bin"))

df

In [ ]:
port = df.group_by(["date", "bin"]).agg(pl.col("ret").mean())
barra_port = df.group_by(["date", "barra_bin"]).agg(pl.col("ret").mean())


port = port.pivot(on="bin", index="date", values="ret")
barra_port = barra_port.pivot(on="barra_bin", index="date", values="ret")


port = port.select(["date"] + labels)
barra_port = barra_port.select(["date"] + labels)


port = port.sort(by="date")
barra_port = barra_port.sort(by="date")

# port
barra_port

In [ ]:
port = port.with_columns((pl.col("0") - pl.col("9")).alias("spread"))
barra_port = barra_port.with_columns((pl.col("0") - pl.col("9")).alias("spread"))

# Unpivot dataframe
port = port.unpivot(index="date", variable_name="bin", value_name="ret")
# barra_port = barra_port.unpivot(index="date", variable_name="barra_bin", value_name="log_spec_ret")
barra_port = barra_port.unpivot(index="date", variable_name="barra_bin", value_name="ret")

# Sort
port = port.sort(by=["date", "bin"])
barra_port = barra_port.sort(by=["date", "barra_bin"])

# port
barra_port

In [ ]:
# Calculate mean, std, sharpe, and tstat of each portfolio

# Mean, std, and count
result = port.group_by("bin").agg(
    [
        pl.col("ret").mean().cast(pl.Float64).alias("mean"),
        pl.col("ret").std().cast(pl.Float64).alias("std"),
        pl.col("ret").count().cast(pl.Float64).alias("count"),
    ]
)

# Sharpe
result = result.with_columns(
    ((pl.col("mean") * 12) / (pl.col("std") * np.sqrt(12)))
    .cast(pl.Float64)
    .alias("annualized_sharpe")
)

# Tstat
result = result.with_columns(
    (pl.col("mean") / (pl.col("std") / pl.col("count").sqrt())).cast(pl.Float64).alias("tstat")
)

# Sort
result = result.sort(by="bin")

# Transpose
result = result.transpose(include_header=True, column_names="bin", header_name="statistic")

print("Monthly results")
result

In [ ]:
barra_port

In [ ]:
# Calculate mean, std, sharpe, and tstat of each portfolio

# Mean, std, and count
result = barra_port.group_by("barra_bin").agg(
    [
        pl.col("ret").mean().cast(pl.Float64).alias("mean"),
        pl.col("ret").std().cast(pl.Float64).alias("std"),
        pl.col("ret").count().cast(pl.Float64).alias("count"),
    ]
)

# Sharpe
result = result.with_columns(
    ((pl.col("mean") * 12) / (pl.col("std") * np.sqrt(12)))
    .cast(pl.Float64)
    .alias("annualized_sharpe")
)

# Tstat
result = result.with_columns(
    (pl.col("mean") / (pl.col("std") / pl.col("count").sqrt())).cast(pl.Float64).alias("tstat")
)

# Sort
result = result.sort(by="barra_bin")

# Transpose
result = result.transpose(include_header=True, column_names="barra_bin", header_name="statistic")

print("Monthly results")
result

In [ ]:
# Create backtest plot

# Log returns
port = port.with_columns(pl.col("ret").log1p().over("bin").alias("logret"))
barra_port = barra_port.with_columns(pl.col("ret").log1p().over("barra_bin").alias("logret"))

# Cummulative sum log returns
port = port.with_columns(pl.col("logret").cum_sum().over("bin").alias("cumret") * 100)
barra_port = barra_port.with_columns(
    pl.col("logret").cum_sum().over("barra_bin").alias("cumret") * 100
)

port

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    port.filter(pl.col("bin").is_in(["0", "9", "spread"])), x="date", y="cumret", hue="bin"
)
plt.xlabel(None)
plt.ylabel("Cummulative Sum Return (%)")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    barra_port.filter(pl.col("barra_bin").is_in(["0", "1", "2", "9", "spread"])),
    x="date",
    y="cumret",
    hue="barra_bin",
)
plt.xlabel(None)
plt.ylabel("Cummulative Sum Return (%)")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    barra_port.filter(pl.col("barra_bin").is_in(["spread"])),
    x="date",
    y="cumret",
    label="Barra Residuals",
)
sns.lineplot(
    port.filter(pl.col("bin").is_in(["spread"])), x="date", y="cumret", label="Raw Returns"
)
plt.legend()
plt.title("Decile Spread Portfolio Performance Comparison")
plt.xlabel(None)
plt.ylabel("Cummulative Sum Return (%)")
plt.show()

## THINGS TO DO:
- Figure out how to merge data to get volume for intuition test
- Test reversal window
- value weight? Optimizer? 
- add features similar to dipesh/miki (rolling sharpe)
